Importing Libraries

For the pyadi-IIO libraries to work we would need libiio (V0.24) bindings to be installed.
Can be installed from the following link: https://github.com/analogdevicesinc/libiio/releases/tag/v0.24

In [1]:
import numpy
from serial import Serial
from time import sleep
from pynput import keyboard
import sys
import select
import os
import csv
import math
from ad7768 import *

Initailization Code

In [2]:
line = 0
writer = 0
run_continuous = False
iterations = 0
data_capture_abort = False
chn_count = 0
inference = 0
data_list = []

def init_data_capture():
    global device
    global data_list
    global chn_count
    global listener
    global samples_block

    ######## User configuration ##########
    # Configure the backend for PC to IIOD interface
    uri = "serial:COM4,230400"  # For UART, baud rate must be same as set in the FW. COM port is physical Or VCOM.
    device_name = "ad7768-1"      # Name of the device must be same as set in the FW.
    ######################################

    # Create an IIO device context
    device = ad7768(uri, device_name)
    device._ctx.set_timeout(100000)

    ######## User configuration ##########
    # Channels to be captured e.g. ["voltage0"]: 1chn, ["voltage0, voltage1"]: 2chns
    device.rx_enabled_channels = ["voltage0"]

    # Note for inference results increment the sample block size +2 so as to accomodate the
    # inference results.
    # The block of samples to be captured. Total samples are received in multiple iterations or blocks
    samples_block = 5122    # The samples needs to be captured in smaller blocks due to limitations
                           # of buffer size (RAM) in the firmware and IIO client timeout factor.
                           # Request to capture samples more than buffer size, will be ignored by firmware.
                           # Large time taken to read the samples from device, may cause timeout on IIO client.
    ######################################

    # Get the channels count from user
    chn_count = len(device.rx_enabled_channels)

    # Store the rx buffer size and rx data type based on input channels
    device.rx_buffer_size = samples_block    # Size of the IIO buffer (buffer is submitted during call to rx() method)
    device._rx_data_type = np.int32     # size of ADC sample
    device._rx_stack_interleaved = True

In [3]:
def init_data_logger():
    global writer
    global chn_count
    file_name = "adc_data_capture.csv"
    current_dir = os.getcwd()
    output_file = os.path.join(current_dir, file_name)
    result_file = open(output_file, 'w', newline="")
    writer = csv.writer(result_file)
    row = []
    # Write the channels list header
    for chn in range(0,chn_count):
        item = "Ch {}".format(chn)
        row.insert(chn, item)
    writer.writerow(row)

In [4]:
def read_buffered_data():
    global line
    global writer
    global device
    global data_arr
    global inference

    # Receive data from device
    data = device.rx()

    if (line == 0):
        print("Data capture started >>")
    print("."*line, end="\r")
    if (chn_count == 1):
        # Convert 1-D array to 2-D array
        data_arr = np.reshape(data, (-1,1))
    else:
        # Transpose data from N-D data array
        data_arr = np.transpose(data)

    inference = data_arr[-2:]
    data_arr = np.resize(data_arr, data_arr.size - 2)
    #r.writerows(data_arr)

    line = line+1
    if (line == 100):
        line = 1
        print("\n", end="\r")

In [5]:
init_data_capture()

In [6]:
init_data_logger()

In [7]:
def do_data_capture():
    global iterations
    global run_continuous
    global data_capture_abort

    iterations = 5
    run_continuous = False
    read_buffered_data()

Run this code to display inference results

In [ ]:
import matplotlib.image as image
import matplotlib.pyplot as plt
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from IPython.display import display, clear_output

im = image.imread("D:/projects/ei_motor_pump/download/ei_img.png")
fig, ax = plt.subplots()
newax = fig.add_axes([0.1,0.9,0.1,0.2])
newax.imshow(im)
newax.axis('off')

while (True):
    do_data_capture()
    fig.suptitle(('\n Inferecence results| Normal = {} % | Bubbling = {} %'.format(int(inference[0]), int(inference[1]))))
    ax.plot(data_arr)
    plt.show()

Run this code for data logging directly into your project

In [ ]:
import json
import time, hmac, hashlib
import requests
import os

#for ecg_contest_final
HMAC_KEY = "Add you HMAC KEY"
API_KEY = "ei_XXX Add you Project API key"

# empty signature (all zeros). HS256 gives 32 byte signature, and we encode in hex, so we need 64 characters here
emptySignature = ''.join(['0'] * 64)
device.rx_buffer_size = 5000

i = 0
while(i < 100):
    do_data_capture()
    data = {
        "protected": {
            "ver": "v1",
            "alg": "HS256",
            "iat": time.time() # epoch time, seconds since 1970
        },
        "signature": emptySignature,
        "payload": {
            "device_name": "ac:87:a3:0a:2d:1b",
            "device_type": "DISCO-L475VG-IOT01A",
            "interval_ms": 0.032,  #Sampling Frequency of the ADC
            "sensors": [
                { "name": "acc", "units": "mV" }
            ],
            "values": data_arr.tolist(),
        }
    }
        
    # encode in JSON
    encoded = json.dumps(data)

    # sign message
    signature = hmac.new(bytes(HMAC_KEY, 'utf-8'), msg = encoded.encode('utf-8'), digestmod = hashlib.sha256).hexdigest()

    # set the signature again in the message, and encode again
    data['signature'] = signature
    encoded = json.dumps(data)

    # and upload the file
    res = requests.post(url='https://ingestion.edgeimpulse.com/api/training/data',
                        data=encoded,
                        headers={
                            'Content-Type': 'application/json',
                            'x-label': 'bubble',
                            'x-file-name': "Normal" + str(i),
                            'x-api-key': API_KEY
                        })
    print(i)
    i  = i + 1

KeyboardInterrupt: 